# A Macroscopically Estimating Model with Endogeneity Considered for Lane-mean Speeds and Speed Deviations (Demo)

## 1 Structuring Data

### 1.1 Import Data

In [1]:
import numpy as np
import pandas as pd
import xlrd
from linearmodels import IV3SLS
import datetime
import math

In [2]:
# flow,speed
file_up_fs = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_up_fs_1.xlsx'
file_do_fs = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_fs_1.xlsx'
# occupancy
file_up_o = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_up_o_1.xlsx'
file_do_o = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_o_1.xlsx'
# truck flow, truck percentage
file_up_tftp = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_up_tftp_1.xlsx'
# file_do_tftp = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_tftp_1.xlsx'
# truck vmt, trauck vht
file_up_vmtvht = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_up_vmtvht_1.xlsx'
# file_do_vmtvht = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_vmtvht_1.xlsx'

data_up_fs = pd.read_excel(file_up_fs)
data_do_fs = pd.read_excel(file_do_fs)
data_up_o = pd.read_excel(file_up_o)
# data_do_o = pd.read_excel(file_do_o)
data_up_tftp = pd.read_excel(file_up_tftp)
# data_do_tftp = pd.read_excel(file_do_tftp)
data_up_vmtvht = pd.read_excel(file_up_vmtvht)
# data_do_vmtvht = pd.read_excel(file_do_vmtvht)

In [3]:
data_up = data_up_fs
data_up = data_up.iloc[:,0:7]
data_up.columns = ['t','f1','s1','f2','s2','f','s']
data_up[['o1','o2']] = data_up_o.iloc[:,[1,2]]
data_up[['tf1','tp1','tf2','tp2']] = data_up_tftp.iloc[:,[1,2,3,4]]
data_up[['vmt1','vht1','vmt2','vht2']] = data_up_vmtvht.iloc[:,[1,2,3,4]]
data_up[['s_do1','s_do2']] = data_do_fs.iloc[:,[2,4]]

###  1.2 Delete Outlier

In [4]:
data_up['t'] = pd.to_datetime(data_up['t'])
data = pd.DataFrame(columns=['t','f1','s1','f2','s2','f','s'])
for day in range(7):
    begin = data_up['t'][0] + datetime.timedelta(days=day)
    end = data_up['t'][0] + datetime.timedelta(days=day+1)
    interval = (data_up['t']>begin) & (data_up['t']<end)
    data_up_t = data_up[interval]
    
    upper_q_s1 = data_up_t['s1'].quantile(0.75)
    lower_q_s1 = data_up_t['s1'].quantile(0.25)
    IQR_s1 = upper_q_s1 - lower_q_s1
    upper_q_s2 = data_up_t['s2'].quantile(0.75)
    lower_q_s2 = data_up_t['s2'].quantile(0.25)
    IQR_s2 = upper_q_s2 - lower_q_s2
    data_up_t = data_up_t[((lower_q_s1-1.5*IQR_s1) <= data_up_t['s1'])& (data_up_t['s1'] <= (upper_q_s1+1.5*IQR_s1))]
    data_up_t = data_up_t[((lower_q_s2-1.5*IQR_s2) <= data_up_t['s2'])& (data_up_t['s2'] <= (upper_q_s2+1.5*IQR_s2))]
    
    data = data.append(data_up_t, sort = True)

data_up = data
data_up.describe()

,o1,o2,s,s1,s2,s_do1,s_do2,tf1,tf2,tp1,tp2,vht1,vht2,vmt1,vmt2
count,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.0,1876.000000,1876.0,1876.000000,1876.0,1876.0,1876.0,1876.000000
mean,3.548188,3.813060,64.340512,65.823881,63.284168,70.563113,64.388486,0.0,0.158849,0.0,0.361194,0.0,0.0,0.0,0.033635
std,1.363698,1.210432,1.180396,1.535113,1.021069,1.331255,1.908361,0.0,0.921640,0.0,2.108254,0.0,0.0,0.0,0.196486
min,0.400000,1.300000,60.100000,61.000000,59.400000,62.700000,57.500000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
25%,2.300000,2.700000,63.900000,65.200000,62.800000,70.500000,63.600000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
50%,3.400000,3.700000,64.400000,66.000000,63.500000,71.100000,64.900000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
75%,4.600000,4.800000,64.800000,66.300000,63.800000,71.200000,65.300000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
max,7.400000,7.300000,68.800000,71.900000,65.900000,73.700000,70.400000,0.0,9.000000,0.0,20.600000,0.0,0.0,0.0,1.900000


### 1.3 Construct Variables

In [5]:
lane1list = ['t']+[col for col in data_up.columns if '1' in col]
data1 = data_up[lane1list]
lane2list = ['t']+[col for col in data_up.columns if '2' in col]
data2 = data_up[lane2list]
variables = ['t','f','o','s','s_do','tf','tp','vht','vmt']
data1.columns = variables
data2.columns = variables
data1.loc[:,'s2'] = data2.loc[:,'s']
data2.loc[:,'s1'] = data1.loc[:,'s']
data1.head()

E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,t,f,o,s,s_do,tf,tp,vht,vmt,s2
1,2017-01-08 00:05:00,21,2.7,66.1,71.1,0.0,0.0,0.0,0.0,63.6
2,2017-01-08 00:10:00,15,2.4,66.2,70.8,0.0,0.0,0.0,0.0,63.7
3,2017-01-08 00:15:00,14,2.3,66.1,71.1,0.0,0.0,0.0,0.0,63.6
4,2017-01-08 00:20:00,17,2.5,66.2,71.1,0.0,0.0,0.0,0.0,63.7
5,2017-01-08 00:25:00,15,2.3,66.2,71.1,0.0,0.0,0.0,0.0,63.7


In [6]:
# use dummy variablesS
for lanedata in [data1, data2]:
    # ln(speed)
    lanedata.loc[:,'lns'] = lanedata['s'].map(lambda x: math.log(x))
    # truck speed
    lanedata.loc[:,'ts'] = 999
    lanedata.loc[lanedata['vht']!=0, 'ts'] =  lanedata['vmt']/lanedata['vht']
#     lanedata['ts'] = lanedata.apply(lambda x: 999.0 if x['vht'] == 0 else x['vmt']/x['vht'], axis = 1) 
#     '''axis : {0 or ‘index’, 1 or ‘columns’}, default 0
#         Axis along which the function is applied:
#         0 or ‘index’: apply function to each column.
#         1 or ‘columns’: apply function to each row.'''
    # traffic flow indicator and high truck flow indicator
    lanedata.loc[:,'fi'] = 0
    lanedata.loc[:,'htfi'] = 0
    lanedata.loc[lanedata['f']<75, 'fi'] = 1
    lanedata.loc[lanedata['tf']>100, 'htfi'] = 1
    # relative truck percentage indicator
    lanedata.loc[:,'tpia'] = 0
    lanedata.loc[:,'tpib'] = 0
    lanedata.loc[(lanedata['tp']>0.6)&(lanedata['f']<50), 'tpia'] = 1
    lanedata.loc[(lanedata['tp']<=0.2)&(lanedata['f']>200), 'tpib'] = 1
    # seasonal indicator (fall will be reference)
    lanedata.loc[:,'seasona'] = 0
    lanedata.loc[:,'seasonb'] = 0
    lanedata.loc[:,'seasonc'] = 0
    lanedata.loc[(2<lanedata['t'].dt.month) & (lanedata['t'].dt.month<6), 'seasona'] = 1 # Spring
    lanedata.loc[(5<lanedata['t'].dt.month) & (lanedata['t'].dt.month<9), 'seasonb'] = 1
    lanedata.loc[(8<lanedata['t'].dt.month) & (lanedata['t'].dt.month<12), 'seasonc'] = 1
    # time-of-week indicator
    lanedata.loc[:,'weeka'] = 0
    lanedata.loc[:,'weekb'] = 0
    lanedata.loc[:,'weekc'] = 0
    lanedata.loc[:,'weekd'] = 0
    lanedata.loc[:,'weeke'] = 0
    lanedata.loc[:,'weekf'] = 0
    lanedata.loc[lanedata['t'].dt.weekday==0, 'weeka'] = 1 # Monday
    lanedata.loc[lanedata['t'].dt.weekday==1, 'weekb'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==2, 'weekc'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==3, 'weekd'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==4, 'weeke'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==5, 'weekf'] = 1
    # time-of-day indicator
    lanedata['hour'] = lanedata['t'].dt.hour
    lanedata.loc[:,'toda'] = 0
    lanedata.loc[:,'todb'] = 0
    lanedata.loc[:,'todc'] = 0
    lanedata.loc[:,'todd'] = 0
    lanedata.loc[(0<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<6), 'toda'] = 1 # midnight to 6 am
    lanedata.loc[(7<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<9), 'todb'] = 1 # am peak
    lanedata.loc[(17<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<19), 'todc'] = 1 # pm peak
    lanedata.loc[(19<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<24), 'todd'] = 1 # night time
    
# ratio of flows in current lane to adjacent lanes
data1['fratio'] = data1['f']/data2['f']
data2['fratio'] = data2['f']/data1['f']

# 2 Equation System

In [7]:
data1[['f','fratio']] = data1[['f','fratio']].infer_objects()
data2[['f','fratio']] = data2[['f','fratio']].infer_objects()
''' Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.
And the error come from other_var.
And after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'
are (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object 
to int and float, respectively.
'''

" Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.\nAnd the error come from other_var.\nAnd after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'\nare (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object \nto int and float, respectively.\n"

In [12]:
np.linalg.matrix_rank(data1[['o','f','ts','fi','fratio']])
''' When running the code block below, another error comming up: Equation u1 instrument array is full rank.
This code line can be used to exam which variables' rank is 1 and which are 0. 
And after excluding the variables rank 0, the code block below can run normally.'''

5

In [9]:
other_var = [ 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd', 'hour']
instruments = ['o', 'f',  'ts', 'fi',  'fratio']
data1.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
data2.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
exo_var = instruments+other_var

u1 = {'dependent': data1[['s1']],
     'exog': data1[other_var],
     'endog': data1[['s2','s_do1']],
     'instruments': data2[instruments]}

u2 = {'dependent': data2[['s2']],
     'exog': data2[other_var],
     'endog': data2[['s1','s_do2']],
     'instruments': data1[instruments]}
equations = dict(u1=u1, u2=u2)
system_3sls = IV3SLS(equations)
system_3sls_res = system_3sls.fit(cov_type='unadjusted')
print(system_3sls_res)

                        System GLS Estimation Summary                         
Estimator:                        GLS   Overall R-squared:              0.9998
No. Equations.:                     2   Cov. Estimator:             unadjusted
No. Observations:                1876   Num. Constraints:                 None
Date:                Wed, Dec 26 2018                                         
Time:                        16:42:26                                         
                                                                              
                                                                              
                     Equation: u1, Dependent Variable: s1                     
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
weeka          0.7428     0.0870     8.5330     0.0000      0.5722      0.9134
weekb         -0.5996     0.0935    -6.4116     0.00